In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1717566354392_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = (SparkSession.builder 
         .master('yarn')
         .getOrCreate())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
s3_bucket_path = "s3://bdcc2024-fp-cpt6/events/*.csv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df = spark.read.csv(s3_bucket_path, sep='\t')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
columns = [
    "GLOBALEVENTID", "SQLDATE", "MonthYear", "Year", "FractionDate",
    "Actor1Code", "Actor1Name", "Actor1CountryCode", "Actor1KnownGroupCode",
    "Actor1EthnicCode", "Actor1Religion1Code", "Actor1Religion2Code",
    "Actor1Type1Code", "Actor1Type2Code", "Actor1Type3Code",
    "Actor2Code", "Actor2Name", "Actor2CountryCode", "Actor2KnownGroupCode",
    "Actor2EthnicCode", "Actor2Religion1Code", "Actor2Religion2Code",
    "Actor2Type1Code", "Actor2Type2Code", "Actor2Type3Code",
    "IsRootEvent", "EventCode", "EventBaseCode", "EventRootCode",
    "QuadClass", "GoldsteinScale", "NumMentions", "NumSources", "NumArticles",
    "AvgTone", "Actor1Geo_Type", "Actor1Geo_FullName", "Actor1Geo_CountryCode",
    "Actor1Geo_ADM1Code", "Actor1Geo_Lat", "Actor1Geo_Long", "Actor1Geo_FeatureID",
    "Actor2Geo_Type", "Actor2Geo_FullName", "Actor2Geo_CountryCode",
    "Actor2Geo_ADM1Code", "Actor2Geo_Lat", "Actor2Geo_Long", "Actor2Geo_FeatureID",
    "ActionGeo_Type", "ActionGeo_FullName", "ActionGeo_CountryCode",
    "ActionGeo_ADM1Code", "ActionGeo_Lat", "ActionGeo_Long", "ActionGeo_FeatureID",
    "DATEADDED", "SOURCEURL"
]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df = df.toDF(*columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
cols = ['GLOBALEVENTID', 'Actor1Code', 'Actor1Name', 'Actor1CountryCode', 'Actor1Type1Code',
 'IsRootEvent', 'EventCode', 'EventBaseCode', 'EventRootCode', 'QuadClass', 'GoldsteinScale',
 'NumMentions', 'NumSources', 'NumArticles', 'AvgTone', 'Year', 'SQLDATE']


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df1 = df.select(cols)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df1.persist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[GLOBALEVENTID: string, Actor1Code: string, Actor1Name: string, Actor1CountryCode: string, Actor1Type1Code: string, IsRootEvent: string, EventCode: string, EventBaseCode: string, EventRootCode: string, QuadClass: string, GoldsteinScale: string, NumMentions: string, NumSources: string, NumArticles: string, AvgTone: string, Year: string, SQLDATE: string]

In [11]:
df1.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df1.columns]).show(truncate=False, vertical=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0----------------------
 GLOBALEVENTID     | 0         
 Actor1Code        | 17946129  
 Actor1Name        | 17945831  
 Actor1CountryCode | 84840261  
 Actor1Type1Code   | 108828253 
 IsRootEvent       | 0         
 EventCode         | 0         
 EventBaseCode     | 0         
 EventRootCode     | 0         
 QuadClass         | 0         
 GoldsteinScale    | 829       
 NumMentions       | 0         
 NumSources        | 0         
 NumArticles       | 0         
 AvgTone           | 0         
 Year              | 0         
 SQLDATE           | 0

In [18]:
test = df1.na.drop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
test.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

22717192

In [22]:
test.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in test.columns]).show(truncate=False, vertical=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0----------------
 GLOBALEVENTID     | 0   
 Actor1Code        | 0   
 Actor1Name        | 0   
 Actor1CountryCode | 0   
 Actor1Type1Code   | 0   
 IsRootEvent       | 0   
 EventCode         | 0   
 EventBaseCode     | 0   
 EventRootCode     | 0   
 QuadClass         | 0   
 GoldsteinScale    | 0   
 NumMentions       | 0   
 NumSources        | 0   
 NumArticles       | 0   
 AvgTone           | 0   
 Year              | 0   
 SQLDATE           | 0

In [23]:
(test.write.parquet('s3://bdcc2024-fp-cpt6/final_parquet', mode='overwrite',
                     partitionBy=['SQLDATE', 'Year']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
tr = spark.read.parquet('s3://bdcc2024-fp-cpt6/final_parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
tr.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

22717192